In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.19.0.2  70.27 KiB  16      63.3%             e39cc177-675f-476a-919d-d9c612aa378c  rack1
UN  172.19.0.4  70.26 KiB  16      59.8%             1a7fef71-6f85-4ad2-8272-7552e4e87b92  rack1
UN  172.19.0.3  70.28 KiB  16      76.9%             ba439315-996e-4531-b1b8-c4f8981a603a  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("DROP KEYSPACE IF EXISTS weather")

cass.execute("""CREATE KEYSPACE weather WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}""")

cass.execute("use weather")
cass.execute("""CREATE TYPE station_record ( tmin int, tmax int)""")

cass.execute("""
CREATE TABLE stations(id text, name text static, date date, record station_record, PRIMARY KEY (id, date)) WITH CLUSTERING ORDER BY (date ASC)""")

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from datetime import datetime
current_date = datetime.now().date()
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

stations_df = spark.read.text("/nb/ghcnd-stations.txt")
stations_processed = stations_df.select(
    expr("substring(value, 1, 11)").alias("ID"),
    expr("substring(value, 39, 2)").alias("STATE"),
    expr("substring(value, 42, 30)").alias("NAME")
)
wi_stations = stations_processed.filter(stations_processed.STATE == "WI")
wi_stations_list = wi_stations.collect()


for station in wi_stations_list:
    cass.execute("""
        INSERT INTO stations (id,name)
        VALUES (%s, %s)
    """,
        (station.ID, station.NAME)
                )
val = cass.execute("""SELECT COUNT(*) FROM stations""")
describe_table_query = "DESCRIBE TABLE stations"
table_description = cass.execute(describe_table_query)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-133edacf-2b4b-426e-88bd-5e21b398ce2d;1.0
	confs: [default]


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central


	found com.typesafe#config;1.4.1 in central


...

	[SUCCESSFUL ] com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1!java-driver-shaded-guava.jar (164ms)
downloading https://repo1.maven.org/maven2/com/typesafe/config/1.4.1/config-1.4.1.jar ...
	[SUCCESSFUL ] com.typesafe#config;1.4.1!config.jar(bundle) (41ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.26/slf4j-api-1.7.26.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.26!slf4j-api.jar (32ms)
downloading https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/4.1.18/metrics-core-4.1.18.jar ...
	[SUCCESSFUL ] io.dropwizard.metrics#metrics-core;4.1.18!metrics-core.jar(bundle) (31ms)
downloading https://repo1.maven.org/maven2/org/hdrhistogram/HdrHistogram/2.1.12/HdrHistogram-2.1.12.jar ...
	[SUCCESSFUL ] org.hdrhistogram#HdrHistogram;2.1.12!HdrHistogram.jar(bundle) (46ms)
downloading https://repo1.maven.org/maven2/org/reactivestreams/reactive-streams/1.0.3/reactive-streams-1.0.3.jar ...


	[SUCCESSFUL ] org.reactivestreams#reactive-streams;1.0.3!reactive-streams.jar (48ms)
downloading https://repo1.maven.org/maven2/com/github/stephenc/jcip/jcip-annotations/1.0-1/jcip-annotations-1.0-1.jar ...
	[SUCCESSFUL ] com.github.stephenc.jcip#jcip-annotations;1.0-1!jcip-annotations.jar (27ms)
downloading https://repo1.maven.org/maven2/com/github/spotbugs/spotbugs-annotations/3.1.12/spotbugs-annotations-3.1.12.jar ...
	[SUCCESSFUL ] com.github.spotbugs#spotbugs-annotations;3.1.12!spotbugs-annotations.jar (36ms)
downloading https://repo1.maven.org/maven2/com/google/code/findbugs/jsr305/3.0.2/jsr305-3.0.2.jar ...
	[SUCCESSFUL ] com.google.code.findbugs#jsr305;3.0.2!jsr305.jar (27ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-query-builder/4.13.0/java-driver-query-builder-4.13.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-query-builder;4.13.0!java-driver-query-builder.jar(bundle) (33ms)
:: resolution report :: resolve 5501ms :: artifacts dl 1393ms


	18 artifacts copied, 0 already retrieved (18067kB/79ms)


23/11/21 02:59:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
#q1
print((cass.execute("describe table stations")).one().create_statement)

CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [6]:
#q2
query = "SELECT name FROM stations WHERE id = 'USW00014837'"
result = cass.execute(query)
name = result.one()[0]
name

'MADISON DANE CO RGNL AP       '

In [7]:
#q3
result = cass.execute("SELECT token(id) FROM stations WHERE id = 'USC00470273'")
station_token = result.one()[0]
station_token

-9014250178872933741

In [8]:
#q4
from subprocess import check_output

node_out = check_output(["nodetool", "ring"]).decode("utf-8")
tokens = []

for line in node_out.split("\n"):
    if 'Normal' in line:
        tok = int(line.split()[-1])  
        tokens.append(tok)
        
tokens.sort()

next_token = None

for tok in tokens:
    if tok > station_token:
        next_token = tok
        break

if next_token is None and tokens:
    next_token = tokens[0]

next_token

-8725289218351686139

In [9]:
#q5
import grpc
import station_pb2
import station_pb2_grpc
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

spark = SparkSession.builder.appName("TemperatureRecords").getOrCreate()

df = spark.read.parquet("./records.parquet")
df_tmax = df.filter(df.element == 'TMAX') \
    .withColumn("date", to_date(col("date").cast("string"), 'yyyyMMdd')) \
    .withColumnRenamed("value", "tmax")

df_tmin = df.filter(df.element == 'TMIN') \
    .withColumn("date", to_date(col("date").cast("string"), 'yyyyMMdd')) \
    .withColumnRenamed("value", "tmin")

df_joined = df_tmax.join(df_tmin, ['station', 'date'])

channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)

for row in df_joined.collect():
    station_id = row["station"]
    date = row["date"].strftime("%Y-%m-%d") 
    tmax = int(row["tmax"])
    tmin = int(row["tmin"])
    request = station_pb2.RecordTempsRequest(
        station=station_id,
        date=date,
        tmin=tmin,
        tmax=tmax
    )
    response = stub.RecordTemps(request)

response = stub.StationMax(station_pb2.StationMaxRequest(station="USW00014837"))

if not response.error:
    max_temp = response.tmax
else:
    max_temp = "Error: " + response.error

max_temp


23/11/21 03:00:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


356

In [10]:
#q6
stationsDF = spark.read.format("org.apache.spark.sql.cassandra").option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3").option("keyspace", "weather").option("table", "stations").load()
stationsDF.createOrReplaceTempView("stations")
tables = spark.catalog.listTables()
tables

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [11]:
#q7
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col

stations_of_interest = ['USW00014839', 'USR0000WDDG', 'USW00014837', 'USW00014898']
df_filtered = df_joined.filter(col('station').isin(stations_of_interest))

df_with_diff = df_filtered.withColumn('temp_diff', col('tmax') - col('tmin'))

avg_diff_by_station = df_with_diff.groupBy('station').agg(avg('temp_diff').alias('avg_temp_diff'))

avg_temp_diff_dict = {row['station']: row['avg_temp_diff'] for row in avg_diff_by_station.collect()}

avg_temp_diff_dict

{'USW00014837': 105.62739726027397,
 'USW00014898': 102.93698630136986,
 'USW00014839': 89.6986301369863,
 'USR0000WDDG': 102.06849315068493}

In [12]:
#q8
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.19.0.2  87.75 KiB  16      100.0%            e39cc177-675f-476a-919d-d9c612aa378c  rack1
UN  172.19.0.4  87.75 KiB  16      100.0%            1a7fef71-6f85-4ad2-8272-7552e4e87b92  rack1
DN  172.19.0.3  87.76 KiB  16      100.0%            ba439315-996e-4531-b1b8-c4f8981a603a  rack1



In [13]:
#q9
channel = grpc.insecure_channel('localhost:5440')
stub = station_pb2_grpc.StationStub(channel)

request = station_pb2.StationMaxRequest(station="USR0000WDDG")

try:
    response = stub.StationMax(request)
    error_content = response.error 
except grpc.RpcError as e:
    error_content = e.details()
print(f"Error field content: {error_content}")

Error field content: need 3 replicas, but only have 2


In [14]:
#q10

stub = station_pb2_grpc.StationStub(channel)

request = station_pb2.RecordTempsRequest(station='blahblahblah',date='2023-04-01',tmin=15,tmax=25)

try:
    response = stub.RecordTemps(request)
    error_message = response.error
except grpc.RpcError as rpc_error:
    error_message = rpc_error.details()

print("error: "+ error_message)

error: 
